In [ ]:
# import libraries
import time
import pandas as pd
from sklearn import set_config
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
import joblib
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
set_config(display="diagram")

In [ ]:
df = pd.read_csv("data/whot_dataset.csv", index_col=0)

In [ ]:
df.info()

In [ ]:
df["Special State"].value_counts()

In [ ]:
X = df.drop(columns="Action")
y = df["Action"]

In [ ]:
# encode target colum
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
 # create pipeline
card_columns = [col for col in df.columns if "Card" in col]
suit_col = ["Requested Suit"]
special_state_col = ["Special State"]

preprocessing = ColumnTransformer([
    ("card_cols", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), card_columns),
    ("suit_col", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), suit_col),
    ("special_col", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), special_state_col)
], remainder="passthrough")

In [ ]:
# train model
start_time = time.time()
model_pipeline = make_pipeline(preprocessing, RandomForestClassifier(n_jobs=-1, random_state=42))
model_pipeline.fit(X, y_encoded)

training_time = time.time() - start_time
print(f"Model training completed in {training_time:.2f} seconds")

In [ ]:
#save model
joblib.dump(model_pipeline, 'model/whot_model.pkl', compress=3)
joblib.dump(label_encoder, 'model/action_encoder.pkl', compress=3)

In [ ]:
sample = pd.DataFrame([{
    'Card 1': 'cross 2',
    'Card 2': 'star 1',
    'Card 3': 'triangle 7',
    'Card 4': 'cross 10',
    'Call Card': 'square 2',
    'Requested Suit': 'NONE',
    'Special State': 'NONE'
}])
pred = model_pipeline.predict(sample)
action = label_encoder.inverse_transform(pred)[0]
print(action)


In [ ]:
# test model on sample
y_train_pred = model_pipeline.predict(X[:200000])

print(classification_report(
    label_encoder.inverse_transform(y_encoded[:200000]),
    label_encoder.inverse_transform(y_train_pred)
))

In [ ]:
print(label_encoder.classes_)